In [ ]:
import pandas as pd
import numpy as np
import numpy.ma as npma
import scipy as sc
from scipy import stats
from scipy.stats import norm, skew #for some statistics
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.kernel_ridge import KernelRidge
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from scipy.special import boxcox1p

from sklearn.linear_model import ElasticNet, Lasso, LassoLars,  BayesianRidge, LassoLarsIC, Ridge
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
os.sys.path.append(os.path.pardir)
from src.models.ConvexRegressor import ConvexRegressor
from src.models.ConvexRegressor import convregparam
from src.models.ConvexRegressor import convregconv

#############################################################################################################
project_dir =os.path.pardir
raw_path = os.path.join(project_dir,'data','processed')
train_path = os.path.join(raw_path, 'train.csv')
test_path = os.path.join(raw_path, 'test.csv')
sns.set()
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
train = train.drop(['Id','Unnamed: 0'],axis = 1)
test = test.drop(['Unnamed: 0'],axis = 1)
y = train['SalePrice'].values
Ids = test['Id'].values
train_x = train.drop('SalePrice',axis = 1).values
test_x = test.drop('Id',axis = 1).values

#################################################################################################################
def LearnAndPredict(x_test,x, y, scaler, solver, ids, q = 0.4):
    yscaler = QuantileTransformer(output_distribution='normal').fit(y.reshape([-1,1]))
    y = yscaler.transform(y.reshape([-1,1])).ravel()
    scaler = scaler.fit(x)
    x = scaler.transform(x)
    solver = solver.fit(x, y)
    x_test = scaler.transform(x_test)
    y = solver.predict(x_test)
    y = yscaler.inverse_transform(y.reshape([-1,1])).ravel()
    df_submission = pd.DataFrame({'Id': ids, 'SalePrice' : y} )
    submission_data_path = os.path.join(os.path.pardir,'data','external')
    submission_file_path = os.path.join(submission_data_path, 'temp.csv')
    df_submission.to_csv(submission_file_path, index=False)
    print(solver.best_score_)
    print(solver.best_params_)
    #sns.distplot(y,fit = norm)
    return y

def linear_ml_solve(x, y, scaler, solver, q = 0.4):
    yscaler = QuantileTransformer(output_distribution='normal').fit(y.reshape([-1,1]))
    y = yscaler.transform(y.reshape([-1,1])).ravel()
    x = scaler.fit_transform(x)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=q, random_state=0)
    solver = solver.fit(x_train, y_train)
    score = solver.score(x_test,y_test)
    print(score)
    y = solver.predict(x_test)
    y = yscaler.inverse_transform(y.reshape([-1,1])).ravel()
    print(solver.best_score_)
    print(solver.best_params_)
    #sns.distplot(y,fit = norm)
    return y

In [ ]:
scaler = RobustScaler()
est1 = Ridge()
est2 = Lasso()
estimators = [est1,est2]
cest = ConvexRegressor(ests=estimators, convparams = {est1.__class__.__name__ : 0.99, est2.__class__.__name__ : 0.01})
param_grid = convregparam(est2,{'alpha': [ 0.02,0.001, 0.00099, 0.009, 0.0008]})
param_grid.update(convregparam(est1,{'alpha' : [0.6,0.5,0.55,0.45]}))
param_grid.update({'convparams' : [convregconv(estimators,[0.01,0.99]), convregconv(estimators,[0.05,0.95])]})
print(param_grid)
gsolver = GridSearchCV(cest, cv=5,param_grid=param_grid)
house_price = LearnAndPredict(test_x,train_x, y, scaler, solver = gsolver, ids = Ids, q = 0.4)